In [2]:
import pandas as pd
import warnings
import fev.utils
import datasets
import fev
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

warnings.simplefilter("ignore")
datasets.disable_progress_bars()

c:\Users\Luca\miniconda3\envs\FairTSBench\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://autogluon.github.io/fev/latest/tutorials/02-dataset-format/

dataset format:
- each dataset entry/row represents a single time series:
    - each entry contains a field of type Sequence(timestamps)
    - at least one field of type Sequence(float) that can be used as the target time series
    - a field that contains the unique ID of each time series
    - each entry can contain arbitrary other fiels like extra dynamic columns (sequence) or static features

In [4]:
#example dataset
ds = datasets.load_dataset("autogluon/chronos_datasets", "monash_kdd_cup_2018", split="train")
ds.set_format("numpy")
pd.DataFrame(ds)

,id,timestamp,target,city,station,measurement
0,T000000,"[2017-01-01T14:00:00.000, 2017-01-01T15:00:00....","[453.0, 417.0, 395.0, 420.0, 453.0, 429.0, 211...",Beijing,aotizhongxin_aq,PM2.5
1,T000001,"[2017-01-01T14:00:00.000, 2017-01-01T15:00:00....","[467.0, 443.0, 467.0, 484.0, 520.0, nan, nan, ...",Beijing,aotizhongxin_aq,PM10
2,T000002,"[2017-01-01T14:00:00.000, 2017-01-01T15:00:00....","[156.0, 143.0, 141.0, 139.0, 157.0, 141.0, 110...",Beijing,aotizhongxin_aq,NO2
3,T000003,"[2017-01-01T14:00:00.000, 2017-01-01T15:00:00....","[7.2, 6.8, 6.9, 7.4, 7.6, 6.5, 3.3, 2.2, 1.3, ...",Beijing,aotizhongxin_aq,CO
4,T000004,"[2017-01-01T14:00:00.000, 2017-01-01T15:00:00....","[3.0, 2.0, 3.0, 3.0, 4.0, 3.0, nan, 4.0, 26.0,...",Beijing,aotizhongxin_aq,O3
...,...,...,...,...,...,...
265,T000265,"[2017-01-01T00:00:01.000, 2017-01-01T01:00:01....","[23.3, 17.5, 16.6, 18.6, 20.4, 20.5, 21.2, 17....",London,BX9,PM2.5
266,T000266,"[2017-01-01T00:00:01.000, 2017-01-01T01:00:01....","[31.7, 29.9, 21.8, 19.1, 23.7, 25.7, 24.4, 20....",London,KC1,PM2.5
267,T000267,"[2017-01-01T00:00:01.000, 2017-01-01T01:00:01....","[23.2, 23.1, 13.9, 12.8, 13.8, 14.9, 15.1, 13....",London,KC1,PM10
268,T000268,"[2017-01-01T00:00:01.000, 2017-01-01T01:00:01....","[21.3, 25.9, 22.6, 23.4, 22.6, 24.9, 27.9, 30....",London,KC1,NO2


Own Dataset

In [9]:
df_edu = pd.read_csv("../possible_datasets/Inequality in Education.csv")
df_edu.drop(columns=["ISO3"], inplace=True)
df_edu.dropna(inplace=True)
df_edu.reset_index(drop=True, inplace=True)

# Encoding
# Ordinal Encoding
df_edu["Human Development Groups"].replace({"Low": 0, "Medium": 1, "High": 2, "Very High": 3}, inplace=True)
# One-Hot Encoding
ohe = OneHotEncoder(sparse_output=False, drop="first")
ohe_arr = ohe.fit_transform(df_edu[["UNDP Developing Regions"]])
ohe_cols = [f"UNDP_Developing_Regions_{cat}" for cat in ohe.categories_[0][1:]]
df_edu = pd.concat([df_edu.drop(columns=["UNDP Developing Regions"]), pd.DataFrame(ohe_arr, columns=ohe_cols)], axis=1)
# z-scoring
scaler = StandardScaler()
df_edu.loc[:, "HDI Rank (2021)":"Inequality in Education (2020)"] = scaler.fit_transform(df_edu.loc[:, "HDI Rank (2021)":"Inequality in Education (2020)"])

list_edu = []
for j in range(len(df_edu)):
    list_edu.append([df_edu.loc[j,f"Inequality in Education ({i})"] for i in range(2010,2021)])
df_edu.drop(columns=df_edu.loc[:, "Inequality in Education (2010)":"Inequality in Education (2020)"].columns, inplace=True)
years = pd.to_datetime([f"{y}-01-01" for y in range(2010, 2021)])  # Timestamp für jedes Jahr (1. Jan)
df_edu["timestamp"] = [np.array(years) for _ in range(len(df_edu))]
df_edu["Inequality in Education"] = [np.array(a) for a in list_edu]
df_edu["target"] = df_edu["Inequality in Education (2021)"]
df_edu.drop(columns=["Inequality in Education (2021)"], inplace=True)
df_edu.rename(columns={"Country": "id"}, inplace=True)

df_edu.to_parquet("../possible_datasets/inequality_education_fev.parquet")
df_edu

,id,Human Development Groups,HDI Rank (2021),UNDP_Developing_Regions_EAP,UNDP_Developing_Regions_ECA,UNDP_Developing_Regions_LAC,UNDP_Developing_Regions_SA,UNDP_Developing_Regions_SSA,timestamp,Inequality in Education,target
0,Afghanistan,0,1.295428,0.0,0.0,0.0,1.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[1.322664579337317, 1.4953222519134515, 1.4852...",45.365170
1,Albania,2,-1.160941,0.0,1.0,0.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-1.0997167805614665, -1.0669027451886566, -1....",12.333440
2,Argentina,3,-1.595696,0.0,0.0,1.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-1.490452246679224, -1.4613997480118872, -1.4...",5.787290
3,Armenia,2,-0.769661,0.0,1.0,0.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-1.7435377670562904, -1.7062258751708705, -1....",2.934950
4,Azerbaijan,2,-0.639234,0.0,1.0,0.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-1.7402330701401953, -1.7026326916248324, -1....",3.565062
...,...,...,...,...,...,...,...,...,...,...,...
86,Venezuela,1,-0.008838,0.0,0.0,1.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-0.6582571581814775, -0.9533372413298422, -0....",8.670590
87,Viet Nam,2,-0.117527,1.0,0.0,0.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-0.7330836523848675, -0.7028309861471114, -0....",15.254790
88,Yemen,0,1.360642,0.0,0.0,0.0,0.0,0.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[1.7365533058373226, 1.7495526160812052, 1.734...",46.137650
89,Zambia,1,0.730246,0.0,0.0,0.0,0.0,1.0,"[2010-01-01T00:00:00.000000000, 2011-01-01T00:...","[-0.17023206968762458, -0.1439115922258987, -0...",20.435720


Try modeling with own dataset

In [ ]:
# Create a task from a dataset stored on Hugging Face Hub
task = fev.Task(
    dataset_path="../possible_datasets/inequality_education_fev.parquet",
    horizon=1,
    num_windows=1,
    known_dynamic_columns=[],
    past_dynamic_columns=[],
    static_columns=["Human Development Groups", "UNDP Developing Regions","HDI Rank (2021)"],
)

In [17]:
task.get_window(0)

: 

In [11]:
# You might need to restart the notebook after installing the dependencies
!pip install -q statsforecast "numpy<=2.2" "scipy<1.16"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fev 0.6.1 requires datasets[s3]<4.0,>=2.15, but you have datasets 4.4.1 which is incompatible.


In [ ]:
from statsforecast.models import AutoETS, SeasonalNaive, Theta


def predict_with_model(task: fev.Task, model_name: str = "seasonal_naive") -> list[datasets.Dataset]:
    assert len(task.target_columns) == 1, "only univariate forecasting supported"
    if model_name == "seasonal_naive":
        model = SeasonalNaive(season_length=task.seasonality)
    elif model_name == "theta":
        model = Theta(season_length=task.seasonality)
    elif model_name == "ets":
        model = AutoETS(season_length=task.seasonality)
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    predictions_per_window = []
    for window in task.iter_windows():
        past_data, future_data = window.get_input_data()
        predictions = [
            {"predictions": model.forecast(y=ts[task.target], h=task.horizon)["mean"]}
            for ts in past_data
        ]
        predictions_per_window.append(datasets.Dataset.from_list(predictions))
    return predictions_per_window

In [ ]:
import time

summaries = []
for model_name in ["seasonal_naive", "ets", "theta"]:
    start_time = time.time()
    predictions_per_window = predict_with_model(task, model_name=model_name)
    infer_time_s = time.time() - start_time
    eval_summary = task.evaluation_summary(
        predictions_per_window,
        model_name=model_name,
        inference_time_s=infer_time_s,
        training_time_s=0.0,
    )

    summaries.append(eval_summary)

In [ ]:
fev.leaderboard(summaries, baseline_model="seasonal_naive")

In [ ]:
fev.pivot_table(summaries, task_columns=["dataset_config"])

In [ ]:
fev.pairwise_comparison(summaries)